In [1]:
#This code demonstartes converting 
#latitude and longitude values from degrees, minutes, seconds, and direction format (GPS data)
#to decimal degrees.
#Import the necessary modules
import re # Searches for a pattern within a string
import pandas as pd
import geopandas as gpd

In [2]:
#Display all rows of a dataframe
pd.set_option("display.max_rows", None)
#Read the dataframe
df = pd.read_csv('lat.csv', sep = ',')

In [3]:
df.head()

,City,Latitude,Longitude
0,Paris,"48°51'26.37""N","2°20'51.773""E"
1,Freiburg,"47°59'37.27""N","7°50'38.32""E"
2,Frankfurt,"50°5'18.696""N","8°39'45.87""E"
3,Berlin,"52°31'0.494""N","13°23'43.50""E"


In [4]:
#Conversion into string data type
lat = df['Latitude'].astype(str)
lng = df['Longitude'].astype(str)

In [5]:
#While loop
#Create two empty lists to append the decimels degrees values
lat_deci = [] #Empty list to store decimal degrees for latitude
lng_deci = [] #Empty list to store decimal degrees for longitude

# Initialize counters
i = 0 
j = 0

while i < len(lat) and j < len(lng):
    #Latitudes
    #Split the latitude column into deg, min, second, direction
    deg, min, second, direction = re.split('[°\'"]+', lat[i])
    # conversion formula
    decimels_lat = (float(deg) + float(min)/60 + float(second)/(60*60)) * (-1 if direction in ['W', 'S'] else 1) #Convert to decimal degrees
    i += 1
    
    #Longitudes
    #Split the longitudes column into deg, min, second, direction
    deg1, min1, second1, direction1 = re.split('[°\'"]+', lng[j])
    # conversion Formula
    decimels_lng = (float(deg1) + float(min1)/60 + float(second1)/(60*60)) * (-1 if direction1 in ['W', 'S'] else 1)
    j += 1
    
    #Create two single-row dataframes
    #Append them to the Empty list
    lats = pd.DataFrame({'Lat_decimals':[decimels_lat]})
    lat_deci.append(lats)
    lngs = pd.DataFrame({'Long_decimals':[decimels_lng]})
    lng_deci.append(lngs)
    
    single_lat = pd.concat(lat_deci, ignore_index=True).reset_index(drop=True)
    single_lng = pd.concat(lng_deci, ignore_index=True).reset_index(drop=True)
    
    frames = [df, single_lat, single_lng]
    
    #Combine dataframes objects horizontally along the x axis by passing in axis = 1
    result = pd.concat(frames, axis = 1).reset_index(drop=True)
    
    #Convert pandasdataframe into Geodataframe
    gdf = gpd.GeoDataFrame(result, 
                           geometry=gpd.points_from_xy(result.Long_decimals, result.Lat_decimals))#,
                           #crs = epsg=4839)

In [6]:
gdf.head()

,City,Latitude,Longitude,Lat_decimals,Long_decimals,geometry
0,Paris,"48°51'26.37""N","2°20'51.773""E",48.857325,2.347715,POINT (2.34771 48.85733)
1,Freiburg,"47°59'37.27""N","7°50'38.32""E",47.993686,7.843978,POINT (7.84398 47.99369)
2,Frankfurt,"50°5'18.696""N","8°39'45.87""E",50.088527,8.662742,POINT (8.66274 50.08853)
3,Berlin,"52°31'0.494""N","13°23'43.50""E",52.516804,13.395417,POINT (13.39542 52.51680)


In [ ]:
#the coordinates are in espg 4326, they can be easily converted to another coordinate system through geopandas
# or any GIS software